In [11]:
%display latex

<h1><p style="color:#16A085"> Álgebra Computacional </p style></h1>
<h2> Índice </h2>

- 1. Algoritmo de Euclides
    - 1.1 Algoritmo de Euclides para un Dominio Euclídeo
    - 1.2 Algoritmo de Euclides extendido para un Dominio Euclídeo
    - 1.3 Ejemplos en $\mathbb{Z}$
    - 1.4 Ejemplos en $\mathbb{Q}[x]$
    - 1.5 Ejemplos en $\mathbb{R}[x]$
    - 1.6 Ejemplos en $\mathbb{Z}[i]$
    

# 1. Algoritmo de Euclides

# 1.1 Algoritmo de Euclides para un Dominio Euclídeo

In [12]:
def euclides(a, b, mod):
    if (b == 0):
        return a
    else:
        return euclides (b, (mod(a, b)), mod)

## 1.2 Algoritmo de Euclides extendido para un Dominio Euclídeo

In [14]:
def extendedEuclides(a, b, div_mod):
    if (b == 0):
        return a, 1, 0 
    else:
        q, r = div_mod(a, b)
        gcd, d, e = extendedEuclides(b, r, div_mod)
        return gcd, e, (d -q * e)

## 1.3 Ejemplos en $\mathbb{Z}$

In [15]:
def mod_int(a, b): return a % b
def div_mod_int(a, b): return a // b, a % b

print(euclides(55, 22, mod_int))
print(extendedEuclides(55, 22, div_mod_int))

11
(11, 1, -2)


## 1.4 Ejemplos en $\mathbb{Q}[x]$

In [22]:
def div_mod_pol(a, b): return a.quo_rem(b);
def mod_pol(a, b): _, r =  div_mod_pol(a, b); return r
    
R.<x> = QQ[]

a = (x^2 +2*x + 1)
b =  (x^2 - 1)

print(euclides(a, b, mod_pol))
print(extendedEuclides(a, b, div_mod_pol))

c = (x+2/3)*(x-3/2)*(x^2+1)
d = (x+2/3)*(x+2/3)*(x^2+1)
print(euclides(c, d, mod_pol)).monic()

2*x + 2
(2*x + 2, 1, -1)
x^3 + 2/3*x^2 + x + 2/3


## 1.5 Ejemplos en $\mathbb{R}[x]$

In [19]:
R.<x> = RR[]
a = (x^2 +2*x + 1)
b =  (x^2 - 1)
print(euclides(a, b, mod_pol))
print(extendedEuclides(a, b, div_mod_pol))

c = (x+2/3)*(x-2/3)*(x^2+1)
d = (x+2/3)*(x+2/3)*(x^2+1)
print(euclides(c, d, mod_pol))

2.00000000000000*x + 2.00000000000000
(2.00000000000000*x + 2.00000000000000, 1.00000000000000, -1.00000000000000)
-3.28954970259306e-17


## 1.6 Ejemplos en $\mathbb{Z}[i]$

In [7]:
def divide_gaussian_integers(z, d):
    r,q = z, 0
    s = 1 if norm(r - d) < norm(r) else -1
    while norm(r - s*d) < norm(r):
        q = q + s
        r = r - s*d

    s = I if norm(r - I*d) < norm(r) else -I
    while norm(r - s*d) < norm(r):
        q = q + s
        r = r - s*d
    return q, r

def mod_gauss(a, b): 
    q,r = divide_gaussian_integers(a, b)
    return r
a = (11 + 2*I) * (3 - 2*I)
b = (11 + 2*I) * (-5 +I)

euclides(a, b, mod_gauss)

11*I - 2

# 2. Teorema chino del resto

In [8]:
def chineseRemainder(coprimes, elems, div_mod, rem):
    m = coprimes[0]
    for i in range(1, len(coprimes)):
        m = m * coprimes[i]
    result = 0;
    for i in range(0, len(coprimes)):
        n = m / coprimes[i]
        _, a, _ = extendedEuclides(n, coprimes[i], div_mod)
        c = rem((a * elems[i]), coprimes[i])
        result = c * n + result
    return result

## 2.1 Ejemplos en $\mathbb{Z}$

In [9]:
def rem(a, b):
    r = abs(a) % abs(b)
    return r if a >= 0 else -r
chineseRemainder([3,5,7],[4,5,33], div_mod_int, rem)

40

## 2.2 Ejemplos en $\mathbb{R}[x]$

In [58]:
R.<x> = RR[]
a = (x + 1)
b =  (x^2 -2*x + 1)
extendedEuclides(a,b,div_mod_pol)
chineseRemainder([a,b],[b,a], div_mod_pol, mod_pol)

0

# 3 MCD en DFU

In [54]:
def gcdDFU(a, b):
    if a.degree() < b.degree():
        return mcdDFU(b,a)
    while b != 0:
        q,r = (b.leading_coefficient()**(a.degree() - b.degree() + 1) * a).quo_rem(b)
        a = b
        b = r
    return a

## 3.1 Ejemplos en $\mathbb{Z}[x]$

In [57]:
R.<x> = ZZ[]
a = (x + 1)**2
b = (x + 1)*(x - 1)
gcdDFU(a, b).monic()

x + 1